In [1]:
%ls

drive/  sample_data/


In [2]:
%cd drive/MyDrive/GEN

/content/drive/MyDrive/GEN


In [3]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

In [4]:
with open('intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# Define model parameters
embed_dim = 16  # Set embedding size to 16
num_heads = 2   # Number of attention heads
ff_dim = 16     # Hidden layer size in feed forward network inside transformer
max_len = 20    # Maximum length of input sequences
vocab_size = 5000  # Update to the actual vocab size

inputs = layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(embed_dim, activation="relu")(x)  # Adjust Dense layer sizes
outputs = layers.Dense(vocab_size, activation="softmax")(x)  # Output vocab_size for next token prediction

model = models.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 token_and_position_embeddi  (None, 20, 16)            80320     
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 20, 16)            2768      
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_2 (Dense)             (None, 16)                272   

In [7]:
print(f"Unique labels: {set(training_labels)}")
print(f"Labels: {labels}")

Unique labels: {'goodbye', 'about', 'complaint', 'name', 'createaccount', 'thanks', 'help', 'greeting'}
Labels: ['greeting', 'goodbye', 'thanks', 'about', 'name', 'help', 'createaccount', 'complaint']


In [8]:
from sklearn.preprocessing import LabelEncoder

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
numerical_labels = lbl_encoder.transform(training_labels)

# Now use numerical_labels in model.fit
epochs = 500
history = model.fit(padded_sequences, np.array(numerical_labels), epochs=epochs)


Epoch 1/500
2/2 [==============================] - 3s 15ms/step - loss: 8.5214 - accuracy: 0.0000e+00
Epoch 2/500
2/2 [==============================] - 0s 11ms/step - loss: 8.4928 - accuracy: 0.0000e+00
Epoch 3/500
2/2 [==============================] - 0s 12ms/step - loss: 8.4670 - accuracy: 0.0000e+00
Epoch 4/500
2/2 [==============================] - 0s 14ms/step - loss: 8.4429 - accuracy: 0.0909
Epoch 5/500
2/2 [==============================] - 0s 11ms/step - loss: 8.4175 - accuracy: 0.0909
Epoch 6/500
2/2 [==============================] - 0s 11ms/step - loss: 8.3910 - accuracy: 0.0909
Epoch 7/500
2/2 [==============================] - 0s 11ms/step - loss: 8.3594 - accuracy: 0.0909
Epoch 8/500
2/2 [==============================] - 0s 11ms/step - loss: 8.3237 - accuracy: 0.0909
Epoch 9/500
2/2 [==============================] - 0s 11ms/step - loss: 8.2861 - accuracy: 0.0909
Epoch 10/500
2/2 [==============================] - 0s 11ms/step - loss: 8.2438 - accuracy: 0.0909
Epoch 1

In [9]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
!pip  install colorama

In [11]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hai
1/1 [==============================] - 0s 163ms/step
ChatBot: My pleasure
User: Hi
1/1 [==============================] - 0s 34ms/step
ChatBot: Hello
User: who r u
1/1 [==============================] - 0s 25ms/step
ChatBot: Any time!
User: quit


In [13]:
!pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━

In [14]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import random
import pickle
import gradio as gr

# Load the intents file
with open("intents.json") as file:
    data = json.load(file)

# Load the trained model
model = keras.models.load_model('chat_model')

# Load the tokenizer object
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the label encoder object
with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

# Parameters
max_len = 20

def predict_response(user_input, history):
    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([user_input]),
                                          truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    # Prepare the response
    bot_response = "I don't understand."
    for i in data['intents']:
        if i['tag'] == tag:
            bot_response = random.choice(i['responses'])

    # Append the new chat messages to history
    history.append((f"You: {user_input}", f"ChatBot: {bot_response}"))
    return history

# Create Gradio interface
with gr.Blocks() as demo:
    # Add custom styling with Markdown
    gr.Markdown("""
    <style>
    .container {
        background-color: #f5f5f5; /* Light gray background color */
        padding: 10px; /* Smaller padding */
        border-radius: 8px; /* Slightly smaller border radius */
        max-width: 400px; /* Narrower width of the container */
        margin: 0 auto; /* Center horizontally */
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        min-height: 100vh; /* Center vertically */
        box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1); /* Optional: Add shadow */
    }
    .input-box {
        width: 100%;
        margin: 5px 0; /* Smaller margin */
    }
    .send-button {
        width: 100%;
        margin: 5px 0; /* Smaller margin */
        display: block;
    }
    </style>
    <div class="container">
        <h2 style="color: #333; font-size: 20px;">Chat with the bot</h2>
    </div>
    """)

    # Chat history display
    chat_history = gr.Chatbot()

    # Input field for user message
    user_input = gr.Textbox(placeholder="Type your message here...", label="Your Message", elem_id="input-box")

    # Send button
    send_button = gr.Button("Send", elem_id="send-button")

    # Define the callback function
    def respond(message, history):
        return predict_response(message, history)

    # Set up interaction
    send_button.click(respond, inputs=[user_input, chat_history], outputs=chat_history)

# Launch the Gradio app
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://599a1e6600bf41fb22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
